## Show SQL Server 2022

In [2]:
SELECT @@VERSION;
GO

(1 row affected)

Total execution time: 00:00:00.003

(No column name)
Microsoft SQL Server 2022 (CTP1.1) - 16.0.101.4 (X64) Dec 15 2021 09:40:31 Copyright (C) 2021 Microsoft Corporation Developer Edition (64-bit) on Windows Server 2022 Datacenter Azure Edition 10.0 <X64> (Build 20348: ) (Hypervisor)


## Catch #1 - DBA Query Catch

After Bob updates the salary of his friend let's see what the ledger says. This query looks at the most recent ledger entries for the Employees table.

In [5]:
USE contosohr;
GO 

SELECT
t.[commit_time] AS [CommitTime] 
, t.[principal_name] AS [UserName]
, l.EmployeeId
, l.[SSN]
, l.[FirstName]
, l.[LastName]
, l.[Salary]
, l.[ledger_operation_type_desc] AS Operation
FROM [dbo].[Employees_Ledger] l
JOIN sys.database_ledger_transactions t
ON t.transaction_id = l.ledger_transaction_id
WHERE t.[commit_time] > DATEADD(MINUTE, -10, GETDATE())
ORDER BY t.commit_time DESC;
GO

Commands completed successfully.

(2 rows affected)

Total execution time: 00:00:00.015

CommitTime,UserName,EmployeeId,SSN,FirstName,LastName,Salary,Operation
2022-01-20 15:31:04.2533333,bob,4,708-44-3627,Jay,Adams,105415.0000,INSERT
2022-01-20 15:31:04.2533333,bob,4,708-44-3627,Jay,Adams,55415.0000,DELETE


## Catch #2 - Valid Application Catch

This query shows in the ledger that the "app" updates Jay's salary but Bob doesn't know we have written the app to audit SQL commands using another ledger table. This table shows that Flynn was the app user who intiated the update.

In [4]:
USE contosohr;
GO 
SELECT
t.[commit_time] AS [CommitTime] 
, t.[principal_name] AS [UserName]
, l.EmployeeId
, l.[SSN]
, l.[FirstName]
, l.[LastName]
, l.[Salary]
, l.[ledger_operation_type_desc] AS Operation
FROM [dbo].[Employees_Ledger] l
JOIN sys.database_ledger_transactions t
ON t.transaction_id = l.ledger_transaction_id
WHERE t.[commit_time] > DATEADD(MINUTE, -1, GETDATE())
ORDER BY t.commit_time DESC;
GO
SELECT [Timestamp]
      ,[UserName]
      ,[Query]
      ,[ledger_start_transaction_id]
      ,[ledger_start_sequence_number]
FROM [ContosoHR].[dbo].[AuditEvents];
GO

Commands completed successfully.

(2 rows affected)

(1 row affected)

Total execution time: 00:00:00.015

CommitTime,UserName,EmployeeId,SSN,FirstName,LastName,Salary,Operation
2022-01-20 15:52:36.2500000,app,4,708-44-3627,Jay,Adams,105415.0000,INSERT
2022-01-20 15:52:36.2500000,app,4,708-44-3627,Jay,Adams,55415.0000,DELETE


Timestamp,UserName,Query,ledger_start_transaction_id,ledger_start_sequence_number
2022-01-20 15:52:55.5566667,Flynn,UPDATE [dbo].[Employees] SET [Salary] = [Salary] + 50000 WHERE [FirstName] = Jay AND [LastName] = Adams,1344,0


## Generate a database digest

In [1]:
USE ContosoHR;
GO
EXECUTE sp_generate_database_ledger_digest;
GO

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.081

latest_digest
"{""database_name"":""ContosoHR"",""block_id"":0,""hash"":""0x04AE03B013961BA736425DB01D4F487D2FF78D0BA5AE828B04A07B4C91485235"",""last_transaction_commit_time"":""2022-01-20T20:28:50.5866667"",""digest_time"":""2022-01-20T20:35:37.6156903""}"


## Check the ledger

Bob has run a stored procedure to supposedly "verify" employees but we are suspicous so will check the employees table and the ledger

In [2]:
USE ContosoHR;
GO
SELECT * FROM Employees;
GO
SELECT
t.[commit_time] AS [CommitTime] 
, t.[principal_name] AS [UserName]
, l.EmployeeId
, l.[SSN]
, l.[FirstName]
, l.[LastName]
, l.[Salary]
, l.[ledger_operation_type_desc] AS Operation
FROM [dbo].[Employees_Ledger] l
JOIN sys.database_ledger_transactions t
ON t.transaction_id = l.ledger_transaction_id
WHERE t.[commit_time] > DATEADD(MINUTE, -10, GETDATE())
ORDER BY t.commit_time DESC;
GO

Commands completed successfully.

(270 rows affected)

(0 rows affected)

Total execution time: 00:00:00.065

EmployeeID,SSN,FirstName,LastName,Salary
1,795-73-9833,Catherine,Abel,61692.0000
2,990-00-6818,Kim,Abercrombie,990.0000
3,009-37-3952,Frances,Adams,5684.0000
4,708-44-3627,Jay,Adams,155415.0000
5,447-62-6279,Robert,Ahlering,49744.0000
6,872-78-4732,Stanley,Alan,38584.0000
7,898-79-8701,Paul,Alcorn,11918.0000
8,561-88-3757,Mary,Alexander,17349.0000
9,904-55-0991,Michelle,Alexander,70796.0000
10,293-95-6617,Marvin,Allen,96956.0000


CommitTime,UserName,EmployeeId,SSN,FirstName,LastName,Salary,Operation


## Catch #3: Catch hidden system table update

Bob thinks he is clever because he has hidden his change but using the ledger I can verify that the hash value that is stored in the digest within SQL Server doesn't match what is in the hash for sys.database\_ledger\_transactions and therefore we have evidence Bob tampered with the ledger.

In [4]:
ALTER DATABASE ContosoHR SET ALLOW_SNAPSHOT_ISOLATION ON;
GO
USE ContosoHR;
GO
DECLARE @json nvarchar(1000)
SET @json = N'{"database_name":"ContosoHR","block_id":0,"hash":"0x04AE03B013961BA736425DB01D4F487D2FF78D0BA5AE828B04A07B4C91485235","last_transaction_commit_time":"2022-01-20T20:28:50.5866667","digest_time":"2022-01-20T20:35:37.6156903"}'
EXEC sp_verify_database_ledger @json;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 37366, Level 16, State 1, Procedure sp_verify_database_ledger, Line 1
The hash computed from sys.database_ledger_transactions for block 0 does not match the hash persisted in sys.database_ledger_blocks.

: Msg 37392, Level 16, State 1, Procedure sp_verify_database_ledger, Line 1
Ledger verification failed.

Total execution time: 00:00:00.083

## Catch #4: Verify using saved digest

Bob has now verified employees again with his new stored procedure. This time the verification shows the saved digest doesn't match the digests stored in system tables. This is a true verification for the disgest saved off in a another "trusted" location.

In [7]:
USE ContosoHR;
GO
DECLARE @json nvarchar(1000)
SET @json = N'{"database_name":"ContosoHR","block_id":0,"hash":"0x04AE03B013961BA736425DB01D4F487D2FF78D0BA5AE828B04A07B4C91485235","last_transaction_commit_time":"2022-01-20T20:28:50.5866667","digest_time":"2022-01-20T20:35:37.6156903"}'
EXEC sp_verify_database_ledger @json;
GO

Commands completed successfully.

: Msg 37368, Level 16, State 1, Procedure sp_verify_database_ledger, Line 1
The hash of block 0 in the database ledger does not match the hash provided in the digest for this block.

: Msg 37392, Level 16, State 1, Procedure sp_verify_database_ledger, Line 1
Ledger verification failed.

Total execution time: 00:00:00.076